# 🗺️ Atelier Analyse de données géographiques

<a href="https://www.linkedin.com/company/data-for-good-grenoble/" target="_blank"><img src="D4G_logo.png" width=200px/></a>

## 🗓️ Jeudi 15 mai 2024 - 18h45 - <a href="https://turbine.coop" target="_blank">La Turbine</a>

## ⚙️ Installation et import des bibliothèques

In [ ]:
%pip install -q nbformat plotly folium mapclassify

In [ ]:
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
import plotly.express as px
import pyodide_http

pyodide_http.patch_all()

## 📚 Liens vers la documentation des bibliothèques

- <a href="https://pandas.pydata.org" target="_blank">Pandas</a>
- <a href="https://geopandas.org/en/stable/" target="_blank">GeoPandas</a>
- <a href="https://matplotlib.org" target="_blank">Matplotlib</a>
- <a href="https://python-visualization.github.io/folium/latest/" target="_blank">Folium</a>
- <a href="https://plotly.com/python/" target="_blank">Plotly</a>

## 🗂️ Sources et formats de données

- [🚌 Navettes saisonnières Transaltitude - Isère](https://transport.data.gouv.fr/datasets/desserte-des-stations-de-ski-iseroises-transaltitude-38)

## 🗺️ Récupération de données GeoJSON

Le projet Github france-geojson propose au format GeoJSON les cartes des **régions, départements, arrondissements, cantons et communes de France (métropole et départements d'outre-mer)** à partir des données publiées par l'<a href="https://www.ign.fr" target="_blank">IGN</a> et l'<a href="https://www.insee.fr/fr/accueil" target="_blank">INSEE</a>. Lien vers le site : https://france-geojson.gregoiredavid.fr

- Récupérer l'url du fichier **GeoJSON** de l'**isère** sur le site france-geojson
- Avec <a href="https://geopandas.org/en/stable/" target="_blank">GeoPandas</a>, insérer les données dans un **GeoDataframe** (à l'aide de la fonction <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html" target="_blank">geopandas.read_file()</a>)
- Afficher ces données sur une **carte** (avec Matplotlib ou Folium) avec **toutes bonnes pratiques de cartographie** (à l'aide de la méthode <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.plot.html" target="_blank">geopandas.GeoDataFrame.plot()</a> ou de la méthode <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html" target="_blank">geopandas.GeoDataFrame.explore()</a> )

In [ ]:
# A COMPLETER

## 🌍 Choix du système de coordonnées de référence (CRS)

La propriété <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.area.html" target="_blank">geopandas.GeoSeries.area</a> permet de récupérer la surface des élèments d'une géométrie.

- Récupérer la **surface du département de l'isère**. Pourquoi cet avertissement ? (aide avec la propiété <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.crs.html" target="_blank">geopandas.GeoDataFrame.crs</a>)
- Changer de système de coordonnées de référence pour passer en **Web Mercator (EPSG:3857)** grâce à la méthode <a href="https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html" target="_blank">geopandas.GeoDataFrame.to_crs()</a>. Est-ce la bonne valeur de la surface ? Pourquoi ?
- Changer de système de coordonnées de référence pour passer en **RGF93 / Lambert-93 (EPSG:2154)**

In [ ]:
# A COMPLETER

## ⛰️ Affichage de l'agglomération de Grenoble

Les **limites des communes de la métropole de Grenoble** peuvent être récupérées à cette adresse au format **GeoJSON** : https://www.data.gouv.fr/fr/datasets/les-communes-de-la-metropole-1/

Comme précédemment :

- **Récupérer** les données dans un **GeoDataframe**
- **Afficher** les données sur une **carte**
- **Calculer la surface** de l'agglomération

In [ ]:
# A COMPLETER

## 🚌 Affichage des navettes saisonnières Transaltitude - Isère

Les **navettes hivernales desservant 14 stations de ski de l’Isère** peuvent être récupérées à cette adresse au format **GeoJSON** : https://transport.data.gouv.fr/datasets/desserte-des-stations-de-ski-iseroises-transaltitude-38

Comme précédemment :

- **Récupérer** les données dans un **GeoDataframe**
- **Afficher** les données sur une **carte**

# A COMPLETER